# Homework 2: Word Similarity

Student Name: Hongyu Chen

Student ID: 1062897

## General info

<b>Due date</b>: Thursday, 4 June 2020 5pm

<b>Submission method</b>: Canvas submission

<b>Submission materials</b>: completed copy of this iPython notebook

<b>Late submissions</b>: -10% per day (both week and weekend days counted)

<b>Marks</b>: 10% of mark for class (with 9% on correctness + 1% on quality and efficiency of your code)

**Note**: As we will be implementing neural networks in this assignment, you're encouraged to build your notebook on **colab**. See the programming exercise in workshop-07 (`10-bert.ipynb`) if you are not familiar with colab.

<b>Materials</b>: See [Using Jupyter Notebook and Python page](https://canvas.lms.unimelb.edu.au/courses/17601/pages/using-jupyter-notebook-and-python?module_item_id=1678430) on Canvas (under Modules>Resources) for information on the basic setup required for this class, including an iPython notebook viewer and the python packages NLTK, Numpy, Scipy, Matplotlib, Scikit-Learn, and Gensim. In particular, if you are not using a lab computer which already has it installed, we recommend installing all the data for NLTK, since you will need various parts of it to complete this assignment. Deep learning libraries such as keras and pytorch are also allowed.  You can also use any Python built-in packages, but do not use any other 3rd party packages (the packages listed above are all fine to use); if your iPython notebook doesn't run on the marker's machine, you will lose marks. <b> You should use Python 3</b>.  

To familiarize yourself with NLTK, here is a free online book:  Steven Bird, Ewan Klein, and Edward Loper (2009). <a href=http://nltk.org/book>Natural Language Processing with Python</a>. O'Reilly Media Inc. You may also consult the <a href=https://www.nltk.org/api/nltk.html>NLTK API</a>.




<b>Evaluation</b>: Your iPython notebook should run end-to-end without any errors in a reasonable amount of time, and you must follow all instructions provided below, including specific implementation requirements and instructions for what needs to be printed (please avoid printing output we don't ask for). You should edit the sections below where requested, but leave the rest of the code as is. You should leave the output from running your code in the iPython notebook you submit, to assist with marking. The amount each section is worth is given in parenthesis after the instructions. 

You will be marked not only on the correctness of your methods, but also the quality and efficency of your code: in particular, you should be careful to use Python built-in functions and operators when appropriate and pick descriptive variable names that adhere to <a href="https://www.python.org/dev/peps/pep-0008/">Python style requirements</a>. If you think it might be unclear what you are doing, you should comment your code to help the marker make sense of it.

<b>Updates</b>: Any major changes to the assignment will be announced via Canvas. Minor changes and clarifications will be announced on the discussion board; we recommend you check it regularly.

<b>Academic misconduct</b>: For most people, collaboration will form a natural part of the undertaking of this homework, and we encourge you to discuss it in general terms with other students. However, this ultimately is still an individual task, and so reuse of code or other instances of clear influence will be considered cheating. We will be checking submissions for originality and will invoke the University’s <a href="http://academichonesty.unimelb.edu.au/policy.html">Academic Misconduct policy</a> where inappropriate levels of collusion or plagiarism are deemed to have taken place.

## Overview

In this homework, you'll be quantifying the similarity between pairs of words of a dataset using different methods with the word co-occurrence in the Brown corpus and synset structure of WordNet. Firstly, you will preprocess the dataset to filter out the rare and ambiguous words. Secondly, you will calculate the similarity scores for pairs of words in the filtered dataset using Lin similarity, NPMI and LSA. Lastly, you will quantify how well these methods work by comparing to a human annotated gold-standard.

## 1. Preprocessing (2 marks)

### Question 1 (1.0 mark)

<b>Instructions</b>: For this homework we will be comparing our methods against a popular dataset of word similarities called <a href="http://alfonseca.org/eng/research/wordsim353.html">Similarity-353</a>. You need to first obtain this dataset, which is on Canvas (assignment 2). The file we will be using is called *set1.tab*. Make sure you save this in the same folder as the notebook.  Except for the header (which should be stripped out), the file is tab formated with the first two columns corresponding to two words, and the third column representing a human-annotated similarity between the two words. <b>You should ignore the subsequent columns</b>.

Here shows the first six lines of the file:

```
Word 1	Word 2	Human (mean)	1	2	3	4	5	6	7	8	9	10	11	12	13	
love	sex	6.77	9	6	8	8	7	8	8	4	7	2	6	7	8	
tiger	cat	7.35	9	7	8	7	8	9	8.5	5	6	9	7	5	7	
tiger	tiger	10.00	10	10	10	10	10	10	10	10	10	10	10	10	10	
book	paper	7.46	8	8	7	7	8	9	7	6	7	8	9	4	9	
computer	keyboard	7.62	8	7	9	9	8	8	7	7	6	8	10	3	9	
```
    
You should load this file into a Python dictionary (NOTE: in Python, tuples of strings, i.e. ("tiger","cat") can serve as the keys of a dictionary to map to their human-annotated similarity). This dataset contains many rare words: we need to filter this dataset in order for it to be better suited to the resources we will use in this assignment. So your first goal is to filter this dataset to generate a smaller test set where you will evaluate your word similarity methods.

The first filtering is based on **document frequencies** (not token frequencies) in the Brown corpus, in order to remove rare words. In this homework, **we will be treating the paragraphs of the Brown corpus as our "documents"**. You can iterate over them by using the `paras` method of the corpus reader. You should remove tokens that are not alphabetic. Tokens should be lower-cased and lemmatized (lemmatize function provided). Store this preprocessed data in *brown_corpus* object (it will be used for question 4 and 5 later).

Now calculate document frequencies for each word type, and use this to remove from your word similarity data any word pairs where at least one of the two words has a document frequency of **$< 8$** in this corpus. You should store all the word pair and similarity mappings in your filtered test set in a dictionary called *filtered_gold_standard*.

Note: the document frequency of a word denotes the number of documents that contains the word.

**Task**: Filter word pairs from *set1.tab* based on document frequencies. Produce *brown_corpus*, which is a list where each element is a set of words for one paragraph (e.g. the first element in *brown_corpus* should contain all the unique word types for the first paragraph). Produce *filtered_gold_standard*, a dictionary of filtered word pairs with human similarity ratings (the dictionary should have (word1, word2) as keys, and similarity ratings as values).

**Check**: Use the assertion statements in *"For your testing"* below for the expected *filtered_gold_standard*.

In [1]:
import nltk
from nltk.corpus import brown
from nltk.corpus import wordnet

nltk.download("brown")
nltk.download("wordnet")

# filtered_gold_standard stores the word pairs and their human-annotated similarity in your filtered test set
filtered_gold_standard = {}

# lemmatizer
lemmatizer = nltk.stem.wordnet.WordNetLemmatizer()

def lemmatize(word):
    lemma = lemmatizer.lemmatize(word,'v')
    if lemma == word:
        lemma = lemmatizer.lemmatize(word,'n')
    return lemma


###
# Your answer BEGINS HERE
###
import re


# check if a word(in lower case) is alphabetic
def pure_word(word):
    if re.match('^[a-z]+$', word):
        return word
    return False


# transform a list of words into lowercase and alphabetic and lemmatized status
def filter_list(array):
    tmp = []
    for w in array:
        lc = w.lower()
        if pure_word(lc):
            tmp.append(lemmatize(lc))
    return tmp


# filter brown corpus
def filter_corpus():
    
    '''
    brown.paras()
    Returns
        the given file(s) as a list of paragraphs, 
        each encoded as a list of sentences, which 
        are in turn encoded as lists of word strings.
    Return type
        list(list(list(str)))
    -- "www.nltk.org/api/nltk.corpus.reader.html"
    '''
    res = []
    
    for para in brown.paras():
        res.append(filter_list(sum(para, [])))
    return res


# get word pair in 'set1.tab'
def get_set1():
    res = []
    with open("set1.tab", 'r') as f:
        for row in f.readlines():
            res.append(row.split()[:3])
    # delete the head line
    res.pop(0)
    
    dic = {}
    for pairs in res:
        dic[(pairs[0], pairs[1])] = pairs[2]
    return dic


# check if a word pair is rare
def not_rare(word_pair, corpus):
    cnt1, cnt2 = 0, 0
    for para in corpus:
        if word_pair[0] in para:
            cnt1 += 1
        if word_pair[1] in para:
            cnt2 += 1
        if cnt1 >= 8 and cnt2 >= 8:
            return True
    return False


brown_corpus = filter_corpus()
set1 = get_set1()
filtered_gold_standard = {}

# delete rare word pairs
for key,val in set1.items():
    if not_rare(key, brown_corpus):
        filtered_gold_standard[key] = float(val)
        

###
# Your answer ENDS HERE
###

print(len(filtered_gold_standard))
print(filtered_gold_standard)


[nltk_data] Downloading package brown to
[nltk_data]     C:\Users\HP\AppData\Roaming\nltk_data...
[nltk_data]   Package brown is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\HP\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


94
{('alcohol', 'chemistry'): 5.54, ('football', 'basketball'): 6.81, ('student', 'professor'): 6.81, ('psychology', 'discipline'): 5.58, ('psychology', 'doctor'): 6.42, ('psychology', 'anxiety'): 7.0, ('tool', 'implement'): 6.46, ('planet', 'sun'): 8.02, ('cup', 'food'): 5.0, ('cup', 'substance'): 1.92, ('tiger', 'cat'): 7.35, ('professor', 'doctor'): 6.62, ('drink', 'ear'): 1.31, ('law', 'lawyer'): 8.38, ('planet', 'galaxy'): 8.11, ('holy', 'sex'): 1.62, ('cup', 'liquid'): 5.9, ('money', 'dollar'): 8.42, ('baby', 'mother'): 7.85, ('drink', 'eat'): 6.87, ('drink', 'mouth'): 5.96, ('psychology', 'depression'): 7.42, ('money', 'property'): 7.57, ('cup', 'drink'): 7.25, ('bird', 'cock'): 7.1, ('journey', 'car'): 5.85, ('movie', 'critic'): 6.73, ('brother', 'monk'): 6.27, ('coast', 'forest'): 3.15, ('money', 'currency'): 9.04, ('cup', 'coffee'): 6.58, ('stock', 'life'): 0.92, ('bishop', 'rabbi'): 6.69, ('journey', 'voyage'): 9.29, ('tiger', 'animal'): 7.0, ('psychology', 'health'): 7.23, 

<b>For your testing: </b>

In [2]:
assert(len(brown_corpus)==15667)
assert(len(filtered_gold_standard) > 50 and len(filtered_gold_standard) < 100)
assert(filtered_gold_standard[('love', 'sex')] == 6.77)

### Question 2 (1.0 mark)

<b>Instructions</b>: Here, you apply the second filtering. The second filtering is based on words with highly ambiguous senses and involves using the NLTK interface to WordNet. Here, you should remove any words which do not have a **single primary sense**. We define single primary sense here as either: (a) having only one sense (i.e. only one synset), or (b) where the count (as provided by the WordNet `count()` method for the lemmas associated with a synset) of the most common sense is at least 4 times larger than the next most common sense. Note that a synset can be associated with multiple lemmas. You should only consider the count of your lemma.

Note: You should lowercase the lemmas of a synset when matching your word; and if there are multiple lowercased lemmas that match your word, you should sum up the count of all matching lemmas.

Additionally, you should remove any words where the primary sense is **not a noun** (this information is also in the synset). Store the synset corresponding to this primary sense in a dictionary for use in the next section.

Given this definition, remove the word pairs from the test set where at least one of the words does not meet the above criteria. When you have applied the two filtering steps, you should store all the word pair and similarity mappings in your filtered test set in a dictionary called *final_gold_standard*.

**Task**: Filter word pairs for any words which do not have a single primary sense and aren't nouns. Produce *final_gold_standard*, a dictionary of filtered word pairs with human similarity ratings. Note: this second filtering is applied on top of the first filtering (question 1). In other words, you shouldn't consider any word pairs that have already been discarded by the first filtering in this question.

**Check**: Use the assertion statements in *"For your testing"* for the expected *final_gold_standard*.

In [3]:
# final_gold_standard stores the word pairs and their human-annotated similarity in your final filtered test set
final_gold_standard = {}
word_primarysense = {} #a dictionary of (word, primary_sense) (used for next section); primary_sense is a synset

###
# Your answer BEGINS HERE
###


def has_single_primary_sense(word):
    syn_dic = {}
    for syn in wordnet.synsets(word):
        # print(syn.name(), syn.lemmas())
        for lem in syn.lemmas():
            if lem.name().lower() == word:
                syn_dic[syn.name()] = syn_dic.get(syn.name(), 0) + lem.count()
    res = sorted(syn_dic.items(), key=lambda item:-item[1])
    if len(res) == 1:
        if res[0][0].split('.')[1] == 'n':
            return wordnet.synsets(word)[0]
    else:
        if res[0][0].split('.')[1] == 'n' and res[0][1] >= res[1][1] * 4:
            for syn in wordnet.synsets(word):
                if syn.name() == res[0][0]:
                    return syn
    return False


for key,val in filtered_gold_standard.items():
    if has_single_primary_sense(key[0]) and has_single_primary_sense(key[1]):
        final_gold_standard[key] = val
        word_primarysense[key[0]] = has_single_primary_sense(key[0])
        word_primarysense[key[1]] = has_single_primary_sense(key[1])
        
###
# Your answer ENDS HERE
###

print(len(final_gold_standard))
print(final_gold_standard)
print(word_primarysense)

26
{('psychology', 'doctor'): 6.42, ('alcohol', 'chemistry'): 5.54, ('football', 'basketball'): 6.81, ('student', 'professor'): 6.81, ('stock', 'egg'): 1.81, ('king', 'queen'): 8.58, ('coast', 'hill'): 4.38, ('psychology', 'mind'): 7.69, ('bread', 'butter'): 6.19, ('coast', 'forest'): 3.15, ('forest', 'graveyard'): 1.85, ('bishop', 'rabbi'): 6.69, ('journey', 'voyage'): 9.29, ('monk', 'slave'): 0.92, ('planet', 'moon'): 8.08, ('psychology', 'health'): 7.23, ('psychology', 'science'): 6.71, ('football', 'tennis'): 6.63, ('coast', 'shore'): 9.1, ('planet', 'galaxy'): 8.11, ('car', 'automobile'): 8.94, ('journey', 'car'): 5.85, ('brother', 'monk'): 6.27, ('baby', 'mother'): 7.85, ('professor', 'doctor'): 6.62, ('money', 'cash'): 9.08}
{'student': Synset('student.n.01'), 'stock': Synset('stock.n.01'), 'journey': Synset('journey.n.01'), 'professor': Synset('professor.n.01'), 'coast': Synset('seashore.n.01'), 'football': Synset('football.n.01'), 'king': Synset('king.n.01'), 'automobile': Syn

<b>For your testing:</b>

In [4]:
assert(len(final_gold_standard) > 10 and len(final_gold_standard) < 40)
assert(final_gold_standard[('professor', 'doctor')] == 6.62)

## 2. Computing word similiarity with Lin similarity, NPMI and LSA (3 marks)

### Question 3 (1.0 mark)

<b>Instructions</b>: Now you will create several dictionaries with similarity scores for pairs of words in your test set derived using the techniques discussed in class. The first of these is the Lin similarity for your word pairs using the information content of the Brown corpus, which you should calculate using the primary sense for each word derived above. You can use the built-in method included in the NLTK interface, you don't have to implement your own. 

When you're done, you should store the word pair and similarity mappings in a dictionary called *lin_similarities*.

**Task**: Compute word pair similarity using Lin similarity for the test set. Produce *lin_similarities*, a dictionary of word pairs (keys) and computed Lin similarity scores (values).

**Check**: Use the assertion statements in *"For your testing"* below for the expected *lin_similarities*. 

In [5]:
from nltk.corpus import wordnet_ic
nltk.download('wordnet_ic')

# lin_similarities stores the word pair and Lin similarity mappings
lin_similarities = {}

###
# Your answer BEGINS HERE
###

brown_ic = wordnet_ic.ic('ic-brown.dat')

for key,_ in final_gold_standard.items():
    lin_similarities[key] = word_primarysense[key[0]].lin_similarity(word_primarysense[key[1]], brown_ic)

###
# Your answer ENDS HERE
###

print(lin_similarities)

[nltk_data] Downloading package wordnet_ic to
[nltk_data]     C:\Users\HP\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet_ic is already up-to-date!


{('psychology', 'doctor'): -0.0, ('alcohol', 'chemistry'): 0.062235427146896456, ('football', 'basketball'): 0.7536025025710653, ('student', 'professor'): 0.26208607023317687, ('stock', 'egg'): -0.0, ('king', 'queen'): 0.25872135992145145, ('coast', 'hill'): 0.5991131628821826, ('psychology', 'mind'): 0.304017384194818, ('bread', 'butter'): 0.711420490146294, ('coast', 'forest'): -0.0, ('forest', 'graveyard'): -0.0, ('bishop', 'rabbi'): 0.6655650900427844, ('journey', 'voyage'): 0.6969176573027711, ('monk', 'slave'): 0.2543108201944307, ('planet', 'moon'): 0.7808882364067532, ('psychology', 'health'): 0.06004979886905243, ('psychology', 'science'): 0.8474590505736942, ('money', 'cash'): 0.7888839126424345, ('coast', 'shore'): 0.9632173804623256, ('planet', 'galaxy'): -0.0, ('car', 'automobile'): 1.0, ('journey', 'car'): -0.0, ('brother', 'monk'): 0.24862817480738675, ('baby', 'mother'): 0.6315913189894092, ('professor', 'doctor'): 0.7036526610448273, ('football', 'tennis'): 0.769995504

<b>For your testing:</b>

In [6]:
assert(lin_similarities[('professor', 'doctor')] > 0.5 and lin_similarities[('professor', 'doctor')] < 1)

### Question 4 (1.0 mark)

**Instructions:** Next, you will calculate Normalized PMI (NPMI) for your word pairs using word frequency derived from the Brown.

PMI is defined as:

\begin{equation*}
PMI = \log_2\left(\frac{p(x,y)}{p(x)p(y)}\right)
\end{equation*}

where

\begin{equation*}
p(x,y) = \frac{\text{Number of paragraphs with the co-occurrence of x and y}}{\sum_i \text{Number of word types in paragraph}_i}
\end{equation*}

\begin{equation*}
p(x) = \frac{\text{Number of paragraphs with the occurrence of x}}{\sum_i \text{Number of word types in paragraph}_i}
\end{equation*}

\begin{equation*}
p(y) = \frac{\text{Number of paragraphs with the occurrence of y}}{\sum_i \text{Number of word types in paragraph}_i}
\end{equation*}

with the sum over $i$ ranging over all paragraphs. Note that there are other ways PMI could be formulated.

NPMI is defined as:

\begin{equation*}
NPMI = \frac{PMI}{-log_2(p(x,y))} = \frac{log_2(p(x)p(y))}{log_2(p(x,y))} - 1
\end{equation*}

Thus, when there is no co-occurrence, NPMI is -1. NPMI is normalized between [-1, +1]. 

You should use the *brown_corpus* object you've created in question 1 as your corpus here. You need to use the basic method for calculating PMI introduced in class (and also in the reading) which is appropriate for any possible definition of co-occurrence (here, there is co-occurrence when a word pair appears in the same paragraph), but you should only calculate PMI for the words in your test set. You must avoid building the entire co-occurrence matrix, instead you should keeping track of the sums you need for the probabilities as you go along. 

When you have calculated NPMI for all the pairs, you should store the word pair and NPMI-similarity mappings in a dictionary called *NPMI_similarities*.

**Task**: Compute word pair similarity using NPMI similarity for the test set. Produce *NPMI_similarities*, a dictionary of word pairs (keys) and computed NPMI similarity scores (values).

**Check**: Use the assertion statements in *"For your testing"* below for the expected *NPMI_similarities*.

In [7]:
# NPMI_similarities stores the word pair and NPMI similarity mappings
NPMI_similarities = {}

###
# Your answer BEGINS HERE
###
import math


def NPML(word_pair):
    w1, w2 = word_pair[0], word_pair[1]
    cnt1, cnt2, cnt_co, word_type = 0, 0, 0, 0
    # one loop is enough (optimazation)
    for para in brown_corpus:
        word_type += len(set(para))
        if w1 in para:
            cnt1 += 1
        if w2 in para:
            cnt2 += 1
        if w1 in para and w2 in para:
            cnt_co += 1
    p_w1 = cnt1 / word_type
    p_w2 = cnt2 / word_type
    p_co = cnt_co / word_type
    
    if p_co == 0:
        return -1
    
    return math.log2(p_w1 * p_w2) / math.log2(p_co) - 1


for key,_ in final_gold_standard.items():
    NPMI_similarities[key] = NPML(key)
    
###
# Your answer ENDS HERE
###

print(NPMI_similarities)

{('psychology', 'doctor'): 0.4613281583726412, ('alcohol', 'chemistry'): 0.6246376972254175, ('football', 'basketball'): 0.7161994042283006, ('student', 'professor'): 0.535495995400926, ('stock', 'egg'): 0.3685128922213745, ('king', 'queen'): 0.4181407297139583, ('coast', 'hill'): 0.33931028834846244, ('psychology', 'mind'): 0.4461666043435182, ('bread', 'butter'): 0.6531272737497535, ('coast', 'forest'): 0.45787396913892886, ('forest', 'graveyard'): -1, ('bishop', 'rabbi'): -1, ('journey', 'voyage'): -1, ('monk', 'slave'): -1, ('planet', 'moon'): 0.658000616232332, ('psychology', 'health'): -1, ('psychology', 'science'): 0.5908740096534864, ('money', 'cash'): 0.44493834723517245, ('coast', 'shore'): 0.5861510629843374, ('planet', 'galaxy'): -1, ('car', 'automobile'): 0.5430334549802616, ('journey', 'car'): -1, ('brother', 'monk'): 0.42993185256931743, ('baby', 'mother'): 0.5149353890388502, ('professor', 'doctor'): -1, ('football', 'tennis'): -1}


<b>For your testing:</b>

In [8]:
assert(NPMI_similarities[('professor', 'doctor')] == -1)

### Question 5 (1.0 mark)

**Instructions:** Here we'll use singular value decomposition (SVD) to derive similarity scores using the Latent Semantic Analysis (LSA) method. Recall that LSA applies SVD and truncation to get a dense vector representation of a word type. To measure similarity between two words we calculate cosine similarity between the LSA vectors of the words.

We'll first build a term-document frequency matrix (as before, a document is a paragraph in Brown corpus). That is, the rows corresponds to words in the vocabulary, and the columns represent the paragraphs/documents in Brown corpus. Each cell records the document frequency of a word (1 if the word appears in the document, 0 otherwise). You should use the *brown_corpus* object created in question 1 as your corpus to build the term-document frequency matrix.

Given the term-document frequency matrix, we'll use `truncatedSVD` in `sklearn` to produce dense vectors of length k = 500, and then use cosine similarity to produce similarities for the word pairs in the test set.

You can use the workshop's notebook on distributional similarity (``09-distributional-similarity.ipynb``) as a starting point, but note that we use term-document frequency matrix (as opposed to the tf-idf matrix in the workshop).

When you are done, you should store the word pair and LSA-similarity mappings in a dictionary called *LSA_similarities*.

**Task**: Compute word pair similarity using LSA. Produce *LSA_similarities*, a dictionary of word pairs (keys) and computed LSA similarity scores (values).

**Check**: Use the assertion statements in *"For your testing"* below for the expected *LSA_similarities*. 



In [35]:
# LSA_similarities stores the word pair and LSA similarity mappings
LSA_similarities = {}

###
# Your answer BEGINS HERE
###
from sklearn.decomposition import TruncatedSVD
from sklearn.feature_extraction.text import CountVectorizer


# get term-document matrix with shape: (term_number, document_number)\
# and term-index dictionary, each term (word) has a unique index such as {w1:0, w2:1, w3:2, ...}
def get_TDmx_TIdic(corpus):
    
    text = []
    for para in corpus:
        # delete duplicate term in one document, to keep the count number always be 1 if target word appears
        para = list(set(para))
        text.append(' '.join(para))
    
    vectorizer = CountVectorizer()
    # X shape: (document_number, term_number)
    X = vectorizer.fit_transform(text)
    
    terms = vectorizer.get_feature_names()
    term_index_dic = {}
    for i in range(len(terms)):
        term_index_dic[terms[i]] = i
        
    return X.T, term_index_dic


# calculate LSA vectors, using SVD
def get_LSA_vectors(term_document_matrix):
    return TruncatedSVD(500).fit_transform(term_document_matrix)


# basic cosine similarity
def cos_sim(v1, v2):
    return np.dot(v1, v2) / (np.linalg.norm(v1) * np.linalg.norm(v2))


term_document_mx, term_index_dic = get_TDmx_TIdic(brown_corpus)
LSA_vectors = get_LSA_vectors(term_document_mx)

print('Length of term_index_dic: ', len(term_index_dic))
print('term_document_matrix shape: ', term_document_mx.shape)
print('LSA_vectors shape: ', LSA_vectors.shape)

for key,_ in final_gold_standard.items():
    index1, index2 = term_index_dic[key[0]], term_index_dic[key[1]]
    LSA_similarities[key] = cos_sim(LSA_vectors[index1], LSA_vectors[index2])

###
# Your answer ENDS HERE
###

print(LSA_similarities)

Length of term_index_dic:  29104
term_document_matrix shape:  (29104, 15667)
LSA_vectors shape:  (29104, 500)
{('psychology', 'doctor'): 0.1094221232687085, ('alcohol', 'chemistry'): 0.10399337275664655, ('football', 'basketball'): 0.24790932327172685, ('student', 'professor'): 0.283735050315131, ('stock', 'egg'): 0.1119896755305052, ('king', 'queen'): 0.13858366869799704, ('coast', 'hill'): 0.2417501247608062, ('psychology', 'mind'): 0.11716790170122621, ('bread', 'butter'): 0.3184431896212829, ('coast', 'forest'): 0.15610240059299932, ('forest', 'graveyard'): 0.06918654171723954, ('bishop', 'rabbi'): 0.0421498947665497, ('journey', 'voyage'): 0.06939319772180737, ('monk', 'slave'): -0.010613374627635128, ('planet', 'moon'): 0.401430706931263, ('psychology', 'health'): 0.0579462942894689, ('psychology', 'science'): 0.2358345587784586, ('money', 'cash'): 0.10567469356441775, ('coast', 'shore'): 0.40905255491719056, ('planet', 'galaxy'): 0.02793245028328546, ('car', 'automobile'): 0.370

<b>For your testing:</b>

In [36]:
assert(LSA_similarities[('professor', 'doctor')] > 0 and LSA_similarities[('professor', 'doctor')] < 0.4)

## Computing word similarity with feedforward language model (3 marks)

### Question 6 (1.0 mark)

**Instructions**: Here we'll build a n-gram neural language model to learn word embeddings, and compute word similarity based on the word embeddings. As before we will use the Brown corpus as training data for our language model, and the first step is to collect the vocabulary.

As before, we'll treat paragraphs in the Brown corpus as our documents. The first 12K paragraphs/documents will serve as our training data, and the rest (3K+ documents) as development data. The first step towards building a language model is to collect the vocabulary, i.e. the set of unique word types in our training data. When collecting the word types, you should lowercase all words, and only keep word types that have a frequency $>= 5$. Store your vocabulary in the _vocab_ object.

Note: 
  - we'll be using _$<$UNK$>$_ to represent unseen words, and so _vocab_ is initialised with the special _$<$UNK$>$_.
  - you do not need to do any other additional preprocessing aside from the aforementioned steps (e.g. no need to remove symbols, etc)
  - you should not use *brown_corpus* here, as you need the words in their original order (*brown_corpus* stores a set of words for each paragraph, and so do not contain word order information)

**Task**: Collect a set of unique word types in the training portion (first 12K paragraphs) of the Brown corpus. Produce _vocab_, which is a set that contains all the word types.

**Check**: Use the assertion statements in *"For your testing"* below to check the vocabulary size.

In [51]:
num_train = 12000
UNK_symbol = "<UNK>"
vocab = set([UNK_symbol])

###
# Your answer BEGINS HERE
###

# get word frequency dictionary
def get_word_fre():
    dic = {}
    for para in brown.paras()[:num_train]:
        for sen in para:
            for w in sen:
                lc = w.lower()
                dic[lc] = dic.get(lc, 0) + 1
    return dic


for key,val in get_word_fre().items():
    if val >= 5:
        vocab.add(key)
        
###
# Your answer ENDS HERE
###

print(len(vocab))

12681


**For your testing:**

In [52]:
assert(len(vocab) > 8000 and len(vocab) < 20000)

### Question 7 (1.0 mark)

**Instructions**: As we'll be building a trigram neural language model (based on lecture 7, page 20), the next step is to collect trigrams to construct our training data. In a trigram neural language model, for example if we have the trigram _cow eats grass_, the input to the model is the first two terms of a trigram (_cow_ and _eats_), and the language model's aim is to predict the last term of the trigram (_grass_). Your task here is to construct the training and development data for the language model. Just like the previous step, the first 12K paragraphs will serve as our training data, and the remaining 3K+ will be for development. You'll need to map words into IDs when constructing the training and development data. Any words that are not in _vocab_ should be mapped to the special _$<$UNK$>$_ symbol.

As an example, given the sentence "_a big fat hungry cow ._", you should create the following training examples:

|input|target|
|:---:|:----:|
|<i>a</i>, _big_|_fat_|
|_big_, _fat_|_hungry_|
|_fat_, _hungry_|_cow_|
|_hungry_, _cow_|_._|


Note:
   - _vocab_ is a set and so does not map words into IDs. You'll need to create a word to ID mapping first based on _vocab_.
   - A trigram should not cross sentence boundary.
   - You should ignore sentences that have less than 3 words (as they are too short to form trigrams).
   - We won't need special starting symbol when collecting the trigrams, as we are only interested in learning word embeddings here (and not computing probabilities of a sentence).


**Task**: Create training and development data. The training input and target data should be stored in the <i>x_train</i> and <i>y_train</i> respectively, and development input and target data in <i>x_dev</i> and <i>y_dev</i> respectively.

**Check**: Use the assertion statements in *"For your testing"* below for the expected shape of the outputs.

In [55]:
###
# Your answer BEGINS HERE
###


# each word will have an ID, stored in vocab_id
list_vocab = list(vocab)
vocab_id = {}
for i in range(len(list_vocab)):
    vocab_id[list_vocab[i]] = i
    
    
# split data and label of a dataset (list of paragraphs in brown.paras())
def split_dataset(dataset):
    x, y = [], []
    for para in dataset:
        for sen in para:
            # too short sentence
            if len(sen) < 3:
                continue
            for start in range(0, len(sen)-2):
                id1 = vocab_id.get(sen[start].lower(), vocab_id["<UNK>"])
                id2 = vocab_id.get(sen[start+1].lower(), vocab_id["<UNK>"])
                id3 = vocab_id.get(sen[start+2].lower(), vocab_id["<UNK>"])
                x.append([id1, id2])
                y.append([id3])
    return np.array(x), np.array(y)
    

train, dev = brown.paras()[:num_train], brown.paras()[num_train:]
x_train, y_train = split_dataset(train)
x_dev, y_dev = split_dataset(dev)

###
# Your answer ENDS HERE
###

    
print(x_train.shape)
print(y_train.shape)
print(x_dev.shape)
print(y_dev.shape)

(872823, 2)
(872823, 1)
(174016, 2)
(174016, 1)


**For your testing:**

In [56]:
assert(x_train.shape[0] == y_train.shape[0])
assert(x_dev.shape[0] == y_dev.shape[0])
assert(x_train.shape[0] > 500000)
assert(x_dev.shape[0] > 50000)

### Question 8 (1.0 mark)

**Instructions**: Now let's build the trigram neural language model. We'll use the language model described in lecture 7 (page 20):

$x' = e(x_1) \oplus e(x_2)$

$h = \tanh(W_1 x' + b)$

$y = $ softmax$(W_2 h)$

where $\oplus$ is the concatenation operation, $x_1$ and $x_2$ are the input words, $e$ is an embedding function, and $y$ is the target word. You can use either `keras` or `pytorch` for building your model.

Set the dimension of the word embeddings and $h$ to 100, and train your model with 3 epochs with a batch size of 256. You will not need to tune hyper-parameters for this task.

After the model is trained, use the word embeddings to compute word similarity in the test set. Store the similarity scores in a dictionary called *lm_similarities*.

Note:
  - For words in the test set that are not in your vocabulary, you should treat them as unknown words. In other words, you should use $<$UNK$>$'s embedding to represent these words.
  - The training may take some time on CPU. You can run your notebook on colab with a GPU if you want faster training (see the programming exercise in workshop-07 if you're not familiar with colab)

**Task**: Train a trigram neural language model, and use the learned word embeddings to compute word similarity. Produce *lm_similarities*, a dictionary that contains word pairs as keys and similarity scores as values.

In [59]:
lm_similarities = {}

###
# Your answer BEGINS HERE
###
# tensorflow 2
import tensorflow as tf
import tensorflow.keras as keras
import random


# get one-hot encoding
def one_hot(y, vocab_size):
    res = []
    for i in range(len(y)):
        tmp = [0] * vocab_size
        tmp[y[i][0]] = 1
        res.append(tmp)
        
    return np.array(res)


# train our model and get trained embeddings
def train_and_get_embedding():
    vocab_size = len(vocab)
    
    # construct the model
    model = keras.Sequential()
    
    # tri-gram -> input_length = 2
    # dimension of the word embeddings set as 100
    # Embedding shape (vocab_size, 100)
    model.add(tf.keras.layers.Embedding(vocab_size, 100, input_length=2))
    
    # concatanate embedding output of two words
    model.add(keras.layers.Flatten())
    
    # h =100
    model.add(tf.keras.layers.Dense(100, activation='tanh'))
    
    model.add(tf.keras.layers.Dense(vocab_size, activation='softmax'))
    
    model.summary()
    
    # compile network
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    
    # train network
    for epoch in range(3):
        # batch size: 256, randomly choose
        rd_idx = np.array(random.sample(range(x_train.shape[0]), 256))
        train_x_batch, train_y_batch = x_train[rd_idx], y_train[rd_idx]
        train_y_batch = one_hot(train_y_batch, vocab_size)
        # print(train_x_batch.shape, train_y_batch.shape)
        model.fit(train_x_batch, train_y_batch, verbose=0)
    
    # first layer
    embeddings = model.layers[0].get_weights()[0]
    
    return embeddings


# embeddings shape (vocab_size, 100)
embeddings = train_and_get_embedding()

for key,_ in final_gold_standard.items():
    # if a word not in vocab (maybe it appears less than 5 times in brown corpus)
    # use UNK_symbol as substitution
    id1, id2 = vocab_id.get(key[0], vocab_id[UNK_symbol]), vocab_id.get(key[1], vocab_id[UNK_symbol])
    assert(len(embeddings[id1]) == 100)
    lm_similarities[key] = cos_sim(embeddings[id1], embeddings[id2])
    
###
# Your answer ENDS HERE
###


print(lm_similarities)

Model: "sequential_15"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_15 (Embedding)     (None, 2, 100)            1268100   
_________________________________________________________________
flatten_10 (Flatten)         (None, 200)               0         
_________________________________________________________________
dense_30 (Dense)             (None, 100)               20100     
_________________________________________________________________
dense_31 (Dense)             (None, 12681)             1280781   
Total params: 2,568,981
Trainable params: 2,568,981
Non-trainable params: 0
_________________________________________________________________
{('psychology', 'doctor'): -0.14107113, ('alcohol', 'chemistry'): -0.032942697, ('football', 'basketball'): -0.08986879, ('student', 'professor'): -0.16301666, ('stock', 'egg'): -0.11510152, ('king', 'queen'): 0.15768078, ('coast', 'hill'): 

## 3. Comparison with the Gold Standard (1 mark)

### Question 9 (1.0 mark)

**Instructions:** Finally, you should compare all the similarities you've created to the gold standard you loaded and filtered in the first step. For this, you can use the Pearson correlation co-efficient (`pearsonr`), which is included in scipy (`scipy.stats`). Be careful converting your dictionaries to lists for this purpose, the data for the two datasets needs to be in the same order for correct comparison using correlation. Write a general function, then apply it to each of the similarity score dictionaries.

When you are done, you should put the result in a dictionary called *pearson_correlations* (use the keys: 'lin', 'NPMI', 'LSA', 'lm').

**Task**: Compute the Pearson correlation coefficient between the estimated similarity scores (Lin, NPMI and LSA similarities) and the gold standard similarity ratings. Produce *pearson_correlations*, a dictionary containing the methods as keys and correlations as values.

**Check**: Use the assertion statements in *"For your testing"* below for the expected *pearson_correlations*. 

<b>Hint:</b> All of the methods used here should be markedly above 0, but also far from 1 (perfect correlation); if you're not getting reasonable results, go back and check your code for bugs! 

In [60]:
from scipy.stats import pearsonr

# pearson_correlations stores the pearson correlations with the gold standard of 'lin', 'NPMI', 'LSA', 'lm'
pearson_correlations = {}

###
# Your answer BEGINS HERE
###

def get_pearsonr(gold_sim, target_sim):
    gold = []
    target = []
    for key,val in gold_sim.items():
        gold.append(val)
        target.append(target_sim[key])
    
    '''
    pearsonr(x, y)
    Parameters
        x Input array.
        y Input array.

    Returns
        r (float) Pearson’s correlation coefficient.
        p-value (float) Two-tailed p-value.
    '''
    return pearsonr(gold, target)[0]


target_sim = ['lin', 'NPMI', 'LSA', 'lm']
for tar in target_sim:
    if tar == 'lin':
        pearson_correlations[tar] = get_pearsonr(final_gold_standard, lin_similarities)
    if tar == 'NPMI':
        pearson_correlations[tar] = get_pearsonr(final_gold_standard, NPMI_similarities)
    if tar == 'LSA':
        pearson_correlations[tar] = get_pearsonr(final_gold_standard, LSA_similarities)
    if tar == 'lm':
        pearson_correlations[tar] = get_pearsonr(final_gold_standard, lm_similarities)

###
# Your answer ENDS HERE
###

print(pearson_correlations)

{'NPMI': 0.18799888869410053, 'lin': 0.5301489978447534, 'LSA': 0.362403020935353, 'lm': 0.2591531391242941}


<b>For your testing:</b>

In [61]:
assert(pearson_correlations['lin'] > 0.4 and pearson_correlations['lin'] < 0.8)

## A final word

Normally, we would not use a corpus as small as the Brown for the purposes of building word vectors. Also, note that filtering our test set to just words we are likely to do well on would typically be considered cheating.